# Problem 1 (fitting quadratics) using a custom HF 🤗 Pipeline

## Step 1: Define a Custom transformers.configuration Configuration for the Model
A custom configuration class is created to define the model's architecture, including input dimension, output dimension, and hidden layers.

In [1]:
from transformers import PreTrainedModel, PretrainedConfig
import torch
import torch.nn as nn


class CustomConfig(PretrainedConfig):
    def __init__(self, input_dim=1, output_dim=1, hidden_dim=16, **kwargs):
        super().__init__(**kwargs)
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim

## Step 2: Create the Custom Model

A simple feedforward neural network is implemented using PyTorch. The model will predict outputs from 2D input data.


In [2]:
class CustomModel(PreTrainedModel):
    config_class = CustomConfig

    def __init__(self, config):
        super().__init__(config)
        self.model = nn.Sequential(
            nn.Linear(config.input_dim, config.hidden_dim),
            nn.ReLU(),
            nn.Linear(config.hidden_dim, config.output_dim)
        )

    def forward(self, x):
        return self.model(x)

# Step 3: Build a Custom Pipeline
A custom Hugging Face pipeline is defined to handle preprocessing, forward pass, and postprocessing for predictions.


In [3]:
from transformers import Pipeline

class CustomPipeline(Pipeline):

    def __init__(self, model, **kwargs):
        super().__init__(model, **kwargs)

    def _sanitize_parameters(self, **kwargs):
        return {}, {}, {}

    def preprocess(self, inputs):
        return torch.tensor(inputs, dtype=torch.float32).unsqueeze(0)

    def _forward(self, model_inputs):
        with torch.no_grad():
            return self.model(model_inputs)

    def postprocess(self, model_outputs):
        return model_outputs.squeeze().tolist()

# Step 4: Instantiate the Model, Config, and Pipeline
The model and pipeline are created based on the defined configuration.


In [4]:
config = CustomConfig(input_dim=1, output_dim=1, hidden_dim=16)

# Check for MPS device support (for Apple silicon)
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

model = CustomModel(config)
# Initialize the model and move it to the correct device
# model = CustomModel(config).to(device)

pipeline = CustomPipeline(model)

Device set to use mps:0


Using device: mps


# Step 5: Train the Model
A simple training loop is used with Mean Squared Error loss and stochastic gradient descent to train the model.

In [5]:
x_train = [[1], [2], [3], [4]]
y_train = [2, 4, 6, 8]

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

for epoch in range(500):
    model.train()
    optimizer.zero_grad()
    inputs = torch.tensor(x_train, dtype=torch.float32).to(device)
    targets = torch.tensor(y_train, dtype=torch.float32).to(device)
    outputs = model(inputs)
    loss = loss_fn(outputs.squeeze(), targets)
    loss.backward()
    optimizer.step()

# Step 6: Test the Pipeline
Use the trained pipeline to make predictions on new data.


In [6]:
#print("dum")
output = pipeline([5])
print(output)

[9.959477424621582]


In [7]:
from scipy.stats import pearsonr

# Test the pipeline against training data
predictions = [pipeline([x])[0] for x in x_train]
targets = [float(y) for y in y_train]

# Calculate correlation coefficient
correlation, _ = pearsonr(predictions, targets)
print(f"Correlation coefficient: {correlation}")

Correlation coefficient: 0.9999605257488521
